In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA
import scipy

In [2]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)

In [3]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"
protien="CPSM_MOUSE"  
time=6
peptides=[x.split('_1_')[0] for x in os.listdir(path) if ((".csv" in x) and ("_1_" in x)) ]
peptides=[x  for x in peptides if 'A' in x]
all_AAs=np.unique([c for c in ("".join(sorted([x.replace('_','').replace('2','').replace('3','') for x in peptides])).lower())])
peptidesinfo=dict()
len(peptides)

24

In [4]:
def charcount(pepSeq):
    res={key: 0 for key in ['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y']}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [5]:
for k in range(len(peptides)):
    data=pd.read_csv(f"C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/{peptides[k]}_{time}_{protien}.csv")
    data["pxt_neh"]=data.NEH*data.pxt
    data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
    minval=min(data.RMSE)
     
    data=data[data.RMSE==minval]
    
    peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                           "charcount":charcount(peptides[k])}
    
    
#     print("========================>",peptidesinfo[peptides[k]]['pxt_neh']/peptidesinfo[peptides[k]]['T_NEH'])



In [6]:
all_AAs

array(['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y'], dtype='<U1')

In [7]:
# visited=set()
# visited.add(peptides[0].split('_')[1])
# coefficients=[]
# for k in range(1,len(peptides)):
#     if(peptides[k].split('_')[1] in visited):
#         continue
        
#     pep1=peptidesinfo[peptides[0]]
#     pep2=peptidesinfo[peptides[k]]    
#     visited.add(peptides[k].split('_')[1])
    
#     r=pep1["pxt_neh"]/pep2["pxt_neh"]
#     coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    
# #     print(r,peptides[0].split('_')[1],peptides[k].split('_')[1])
#     if(len(coefficients)==len(all_AAs)-1):
#         break
# print(len(coefficients))


# if(len(coefficients)<len(all_AAs)-1):
#     for k in range(len(peptides)-2,1,-1):
#         pep1=peptidesinfo[peptides[len(peptides)-1]]
#         pep2=peptidesinfo[peptides[k]]
#         r=pep1["pxt_neh"]/pep2["pxt_neh"]
#         coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
#         print(r)
#         if(len(coefficients)==len(all_AAs)-1):
#             break


In [12]:
visited=set()
visited.add(peptides[0].split('_')[1])
coefficients=[]
for k in range(0,len(peptides)-1):
    if(peptides[k+1].split('_')[1] in visited):
        continue
        
    pep1=peptidesinfo[peptides[k]]
    pep2=peptidesinfo[peptides[k+1]]    
    visited.add(peptides[k].split('_')[1])
    
    r=pep1["pxt_neh"]/pep2["pxt_neh"]
    coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    
#     print(r,peptides[0].split('_')[1],peptides[k].split('_')[1])
#     if(len(coefficients)==len(all_AAs)-1):
#         break
print(len(coefficients))


# if(len(coefficients)<len(all_AAs)-1):
#     for k in range(len(peptides)-2,1,-1):
#         pep1=peptidesinfo[peptides[len(peptides)-1]]
#         pep2=peptidesinfo[peptides[k]]
#         r=pep1["pxt_neh"]/pep2["pxt_neh"]
#         coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
#         print(r)
#         if(len(coefficients)==len(all_AAs)-1):
#             break


23


In [13]:
coefficients=np.array(coefficients)
coefficients[:,1:].shape

(23, 19)

In [14]:
# a_val=4
# coefficients=np.array(coefficients)
# # sol=list(np.linalg.lstsq(coefficients[:,1:],-a_val*coefficients[:,0])[0]) 
# sol=list(np.linalg.solve(coefficients[:,1:],-a_val*coefficients[:,0])) 
# sol.insert(0,a_val)
# sol

In [17]:
a_val=4
sol=list(scipy.optimize.nnls(coefficients[:,1:],-a_val*coefficients[:,0])[0])
sol.insert(0,a_val)
len(sol)

20

In [18]:

t_neh,e_neh=[],[]
for i in range (len(peptides)):
    pep1=peptidesinfo[peptides[i]]
    p_neh=0
    for index,c in enumerate(all_AAs):
#         print(c,pep1["charcount"][c],pep1["charcount"][c]*sol[index])
        p_neh+=(pep1["charcount"][c]*sol[index])
#     print("\n",p_neh,"\t",pep1["T_NEH"])
    t_neh.append(pep1["T_NEH"])
    e_neh.append(p_neh)
    
res=pd.DataFrame([peptides,t_neh,e_neh])
res=res.transpose()
res.columns=["peptides","True_NEH","e_NEH"]
res["dif"]=abs(res.True_NEH-res.e_NEH)
print(np.mean(res.dif),np.median(res.dif),np.std(res.dif))
res.sort_values(by='dif')

7.361900467705838 6.40736136028621 3.0306761276185954


,peptides,True_NEH,e_NEH,dif
18,_TAHIVLEDGTK_3,17.0,14.885561,2.114439
5,_FLGVAEQLHNEGFK_2,27.0,23.187013,3.812987
6,_FLGVAEQLHNEGFK_3,27.0,23.187013,3.812987
0,_AADTIGYPVMIR_2,22.0,18.138174,3.861826
1,_AFAISGPFNVQFLVK_2,25.0,20.751143,4.248857
16,_SIFSAVLDELK_2,18.0,12.574707,5.425293
9,_IALGIPLPEIK_2,19.0,13.52826,5.47174
2,_AFAmTNQILVER_2,25.0,19.324708,5.675292
13,_QAFITAQNHGY_2,24.0,18.321511,5.678489
11,_IASVQTNEVGLK_2,21.0,15.096246,5.903754


In [101]:
temp=pd.DataFrame(coefficients)
temp.columns=all_AAs
temp 

,a,c,d,e,f,g,h,i,k,l,m,n,p,q,r,s,t,v,w,y
0,0.247619,0.000000,1.000000,0.000000,-2.628572,0.123809,0.000000,1.123809,-0.876191,-0.876191,1.000000,-0.876191,0.123809,-0.876191,1.000000,-0.876191,1.000000,-0.752381,0.000000,1.000000
1,0.257577,0.000000,1.000000,-0.871212,-0.871212,1.000000,0.000000,1.128788,0.000000,-0.871212,0.128788,-0.871212,1.000000,-0.871212,0.128788,0.000000,0.128788,0.128788,0.000000,1.000000
2,-0.072071,0.000000,0.309310,0.000000,0.000000,-1.072071,0.000000,0.618619,0.000000,0.000000,1.000000,-1.381381,-0.381381,0.000000,0.309310,0.000000,-0.381381,1.000000,0.000000,1.000000
3,-0.123077,0.000000,1.000000,-1.415385,-0.707692,1.000000,0.000000,2.000000,0.000000,-1.415385,1.000000,-2.123077,1.000000,0.000000,1.000000,0.000000,0.292308,0.292308,0.000000,1.000000
4,1.248366,0.000000,1.000000,-1.503267,-1.503267,-0.503267,-0.751634,2.000000,-0.751634,-1.503267,1.000000,-0.751634,1.000000,-0.751634,1.000000,0.000000,1.000000,0.248366,0.000000,1.000000
5,1.304611,-0.695389,0.304611,-0.695389,-0.695389,0.304611,-0.695389,0.609221,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.304611,-2.781558,1.000000,-0.390779,-0.695389,1.000000
6,1.259260,-0.740740,1.000000,-0.740740,-0.740740,-0.481481,-1.481481,0.518519,0.000000,-1.481481,1.000000,-0.740740,0.259260,-0.740740,1.000000,0.000000,0.259260,0.259260,0.000000,1.000000
7,0.826531,0.000000,1.000000,-1.173469,0.000000,-0.173469,0.000000,-1.520406,-1.173469,-2.346937,1.000000,0.000000,-1.346937,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
8,0.057705,0.000000,0.352568,-1.294863,-0.647432,1.000000,0.000000,1.352568,-0.647432,-0.647432,0.352568,0.000000,0.352568,0.000000,1.000000,-1.294863,1.000000,0.352568,0.000000,1.000000
9,1.061224,0.000000,1.000000,-0.938776,0.000000,0.061224,0.000000,1.061224,-0.938776,-0.938776,1.000000,-0.938776,1.000000,-0.938776,1.000000,-0.938776,0.061224,-0.877551,0.000000,1.000000


In [25]:
for c in range( len(coefficients)):
    print(np.dot(coefficients[c],sol))

-2.2737367544323206e-13
1.1368683772161603e-13
-4.547473508864641e-13
1.0231815394945443e-12
2.2737367544323206e-13
5.684341886080801e-13
6.821210263296962e-13
-2.2737367544323206e-13
0.0
-1.1368683772161603e-13
6.931641366400732e-13
-2.5874815803587202e-14
3.0027345616533543e-13
1.727332634446871e-13
6.821210263296962e-13
7.227478695623776e-13
2.2737367544323206e-13
0.0
2.4725778969638176e-13
